# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
weathercity_df = "output/weather_cities.csv"
part1_df = pd.read_csv(weathercity_df)
part1_df

City  Cloudiness Country        Date  Humidity      Lat  \
0      port alfred         100      ZA  1619666386        86 -33.5906   
1     punta arenas          75      CL  1619666177        87 -53.1500   
2         baherden         100      TM  1619666387        56  38.4362   
3           kodiak          90      US  1619666253        87  57.7900   
4          ushuaia          75      AR  1619666252        80 -54.8000   
..             ...         ...     ...         ...       ...      ...   
535          olhao           0      PT  1619666533        88  37.0286   
536  prince rupert          90      CA  1619666533        93  54.3161   
537   mendeleyevsk           0      RU  1619666533        93  55.9082   
538        caceres          95      ES  1619666534        89  39.5220   
539         sorong          11      ID  1619666534        78  -0.8833   

          Lng  Max Temp  Wind Speed  
0     26.8910     67.37       15.77  
1    -70.9167     39.20       16.11  
2     57.4316     72.28        5.37  
3   -152.4072     42.80        9.22  
4    -68.3000     35.60       31.07  
..        ...       ...         ...  
535   -7.8411     55.40        4.61  
536 -130.3201     48.20       11.50  
537   52.2905     37.40       15.66  
538   -6.3748     48.99        5.01  
539  131.2500     82.24        3.94  

[540 rows x 9 columns]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [14]:
# Create a map using coordinates 
marker_locations = part1_df[['Lat', 'Lng']]
weights = part1_df['Humidity'].astype(float)
# Plot Heatmap
fig = gmaps.figure(zoom_level = 2, center = [0,0])

# Create heat layer
fig.add_layer(gmaps.heatmap_layer(marker_locations,  weights= weights,
                                 dissipating=False, max_intensity=100,
                                 point_radius=3))

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
#Fitting my weather criteria:
high_temp = 85
low_temp = 70
new_df = part1_df.loc[(part1_df['Max Temp']< high_temp) & (part1_df['Max Temp'] > low_temp)        
                                                        & (part1_df['Cloudiness'] <= 10)
                                                        & (part1_df['Wind Speed'] < 24)
                                                        & (part1_df['Humidity'] > 30)
                                                        & (part1_df['Humidity'] < 70),:]        
new_df

City  Cloudiness Country        Date  Humidity      Lat  \
63           poum           0      NC  1619666402        51 -20.2333   
156     kavaratti           6      IN  1619666429        68  10.5669   
221  port lincoln          10      AU  1619666234        39 -34.7333   
231        noumea           0      NC  1619666338        53 -22.2763   
238           voh           0      NC  1619666450        48 -20.9667   
280     carnarvon           0      AU  1619666219        51 -24.8667   
323      waingapu           9      ID  1619666474        51  -9.6567   
395  port augusta           1      AU  1619666347        33 -32.5000   
454      xuanzhou          10      CN  1619666510        41  30.9525   

          Lng  Max Temp  Wind Speed  
63   164.0167     75.69        4.12  
156   72.6420     84.33        3.13  
221  135.8667     71.35        3.56  
231  166.4572     78.80       18.41  
238  164.7000     79.34       10.67  
280  113.6333     82.40       10.36  
323  120.2641     83.19        4.99  
395  137.7667     75.99        8.05  
454  118.7553     76.57       16.67

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df = pd.DataFrame(new_df)
hotel_df['Hotel Name'] = ''
hotel_df

City  Cloudiness Country        Date  Humidity      Lat  \
63           poum           0      NC  1619666402        51 -20.2333   
156     kavaratti           6      IN  1619666429        68  10.5669   
221  port lincoln          10      AU  1619666234        39 -34.7333   
231        noumea           0      NC  1619666338        53 -22.2763   
238           voh           0      NC  1619666450        48 -20.9667   
280     carnarvon           0      AU  1619666219        51 -24.8667   
323      waingapu           9      ID  1619666474        51  -9.6567   
395  port augusta           1      AU  1619666347        33 -32.5000   
454      xuanzhou          10      CN  1619666510        41  30.9525   

          Lng  Max Temp  Wind Speed Hotel Name  
63   164.0167     75.69        4.12             
156   72.6420     84.33        3.13             
221  135.8667     71.35        3.56             
231  166.4572     78.80       18.41             
238  164.7000     79.34       10.67             
280  113.6333     82.40       10.36             
323  120.2641     83.19        4.99             
395  137.7667     75.99        8.05             
454  118.7553     76.57       16.67

In [ ]:
# find the closest hotel of each type to coordinates
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#setting parameters
params = {
        "type": "hotel",
        "radius": target_radius,
        "key": g_key
         }
# Use the lat/lng recovered to identify hotels
for index, row in hotel_df.iterrows():
    # Get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    # Change locations
    marker_location = hotel_df[['Lat', 'Lng']]
    #Modify parameters 
    params["location"] = marker_location
    # Make API request and convert to json
    response_hotel = requests.get(base_url, params=params).json()
    # Print 
    response_hotel
    try:
    # Handle exceptions
    except:
    pass



# run a request using our params dictionary
response = requests.get(base_url, params=params)
response
# convert response to json
#places_data = response.json()
# Print the json (pretty printed)
#print(json.dumps(places_data, indent=4, sort_keys=True))

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
